In [1]:
import os
LANGCHAIN_API_KEY = os.getenv(key="LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv(key="LANGCHAIN_ENDPOINT")
LANGCHAIN_TRACING_V2 = os.getenv(key="LANGCHAIN_TRACING_V2")

In [2]:
import bs4
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings

In [3]:
# define llm and embedding model to be used
llm = Ollama(model="phi",temperature=0,timeout=300)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = HuggingFaceEmbeddings(model_name=model_name)

In [4]:
def data_ingestion():
    import bs4
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.document_loaders import WebBaseLoader
    from langchain_community.vectorstores import Chroma
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain.storage import LocalFileStore
    from langchain.embeddings import CacheBackedEmbeddings

    fs = LocalFileStore("./cache/")

    # Caching
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embed_model = HuggingFaceEmbeddings(model_name=model_name)
    fs = LocalFileStore("./cache/")
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embed_model, fs, namespace=model_name
    )

    # Load Documents
    loader = WebBaseLoader(
        web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    docs = loader.load()

    # Split Data
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    splits = text_splitter.split_documents(documents=docs)

    # create vectorstore
    vectorstore = Chroma.from_documents(documents=splits, embedding=cached_embedder)
    return vectorstore

    

In [5]:
vectorstore = data_ingestion()

In [6]:
vectorstore.similarity_search("What is Task Decomposition?")

[Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Task Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='(3) Task execution: Expert models execute on the specific tasks and log results.\nInstructi

In [13]:
retriever = vectorstore.as_retriever()
retriever.invoke("What is Task Decomposition?")

[Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Task Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='(3) Task execution: Expert models execute on the specific tasks and log results.\nInstructi

In [10]:
#### RETRIEVAL and GENERATION ####
# Prompt
prompt = hub.pull("rlm/rag-prompt")
llm = llm

In [11]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    |StrOutputParser()
)

In [19]:
print(rag_chain.invoke("What is Task Decomposition?"))

 Task decomposition refers to breaking down a complex task into smaller, more manageable subgoals or steps that can be accomplished in a logical sequence. This technique is often used with language models like LLM (Language Model) to improve their performance on tasks such as question-answering. 

There are three ways of performing task decomposition: 1) using simple prompting techniques, 2) providing task-specific instructions, and 3) incorporating human inputs. The first method involves giving the model a prompt that outlines the steps required to complete the task. The second method requires the user to provide specific instructions for the model to follow. The third method involves asking humans to decompose the task into smaller subgoals or steps. 

Chain of thought (CoT) is a popular prompting technique used in task decomposition that helps models break down complex tasks into simpler ones by guiding them through a step-by-step process. This approach has been shown to improve mod

In [15]:
def data_ingestion_in_memory():
    import bs4
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.document_loaders import WebBaseLoader
    from langchain_community.vectorstores import Chroma
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain.storage import LocalFileStore, InMemoryByteStore
    from langchain.embeddings import CacheBackedEmbeddings

    store = InMemoryByteStore()

    # Caching
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embed_model = HuggingFaceEmbeddings(model_name=model_name)
    fs = LocalFileStore("./cache/")
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embed_model, store, namespace=model_name
    )

    # Load Documents
    loader = WebBaseLoader(
        web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    docs = loader.load()

    # Split Data
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    splits = text_splitter.split_documents(documents=docs)

    # create vectorstore
    vectorstore = Chroma.from_documents(documents=splits, embedding=cached_embedder)
    return vectorstore

In [16]:
vectorstore_in_memory = data_ingestion_in_memory()

In [18]:
vectorstore_in_memory.similarity_search("What is Task Decomposition?")

[Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Task Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tas

In [20]:
retriever_mem = vectorstore_in_memory.as_retriever()
retriever_mem.invoke("What is Task Decomposition?")

[Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Task Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tas

In [21]:
#### RETRIEVAL and GENERATION ####
# Prompt
prompt = hub.pull("rlm/rag-prompt")
llm = llm

In [23]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain_mem = (
    {"context": retriever_mem | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    |StrOutputParser()
)

In [24]:
print(rag_chain_mem.invoke("What is Task Decomposition?"))

 Task decomposition refers to breaking down a complex task into smaller, more manageable subgoals or subtasks. This can be achieved through various techniques such as using language models with simple prompts, providing task-specific instructions, or incorporating human inputs. The goal is to simplify the task and make it easier for the model to understand and execute.

